<a href="https://colab.research.google.com/github/gaganreddy7977/Datascience-python-/blob/main/Resume_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Pinecone

In [ ]:
!pip install sentence-transformers

In [ ]:
pip install pypdf

In [ ]:
import os
import time
from pypdf import PdfReader
from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion, Metric
from sentence_transformers import SentenceTransformer

# ---------------------------------------------------------
# Load Embedding Model
# ---------------------------------------------------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embedding_dim = model.get_sentence_embedding_dimension()
print("Embedding dimension:", embedding_dim)

# ---------------------------------------------------------
# Initialize Pinecone
# ---------------------------------------------------------
api_key = "pcsk_2Vhtvh_MAEVCuKtofeqNCXDpxFyCi1tgvWBtfrpWNV55VAwMudfyFzC5JQyWCLJL6JqbtF"   # ⚠️ move to env variable in real projects
pc = Pinecone(api_key=api_key)

index_name = "resume-search-index"

# Delete index if exists
if pc.has_index(index_name):
    pc.delete_index(index_name)

# Create index
pc.create_index(
    name=index_name,
    dimension=embedding_dim,
    metric=Metric.COSINE,
    spec=ServerlessSpec(
        cloud=CloudProvider.AWS,
        region=AwsRegion.US_EAST_1
    )
)

index = pc.Index(host=pc.describe_index(index_name).host)
print("Index created successfully")

# ---------------------------------------------------------
# PDF Text Extraction
# ---------------------------------------------------------
pdf_folder_path = r"/content/drive/MyDrive/Resume Database"

def extract_text_from_pdfs(folder_path):
    documents = {}
    doc_id = 1

    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, file_name)
            reader = PdfReader(file_path)

            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "

            if text.strip():
                documents[f"doc_{doc_id}"] = text
                doc_id += 1

    return documents

start_time = time.time()
documents = extract_text_from_pdfs(pdf_folder_path)
end_time = time.time()

print(f"Text extraction completed in {end_time - start_time:.2f} seconds")
print(f"Total documents loaded: {len(documents)}")

# ---------------------------------------------------------
# Generate Embeddings
# ---------------------------------------------------------
doc_ids = list(documents.keys())
doc_texts = list(documents.values())

embeddings = model.encode(
    doc_texts,
    batch_size=16,
    show_progress_bar=True
).tolist()

vectors = list(zip(doc_ids, embeddings))

# ---------------------------------------------------------
# Upsert into Pinecone
# ---------------------------------------------------------
index.upsert(vectors=vectors)

# ---------------------------------------------------------
# Wait Until Indexing Completes
# ---------------------------------------------------------
def wait_until_indexing_complete(idx, expected_count, check_interval=5):
    while True:
        stats = idx.describe_index_stats()
        current_count = stats.total_vector_count
        print(f"Indexed: {current_count}/{expected_count}")
        if current_count >= expected_count:
            break
        time.sleep(check_interval)

wait_until_indexing_complete(index, len(documents))

# ---------------------------------------------------------
# Semantic Queries
# ---------------------------------------------------------
query_text1 = "data engineering resume, azure data factory, azure databricks"
query_text2 = "data science machine learning langchain gen ai agentic ai"

query_embedding = model.encode(query_text1).tolist()

results = index.query(
    vector=query_embedding,
    top_k=5,
    include_values=False
)

print("\nSearch Results:")
print(results)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384
Index created successfully
Text extraction completed in 7.44 seconds
Total documents loaded: 213


Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Indexed: 213/213

Search Results:
QueryResponse(matches=[{'id': 'doc_154', 'score': 0.460328102, 'values': []}, {'id': 'doc_71', 'score': 0.413956642, 'values': []}, {'id': 'doc_26', 'score': 0.393188477, 'values': []}, {'id': 'doc_110', 'score': 0.385479927, 'values': []}, {'id': 'doc_50', 'score': 0.379592896, 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 28 Jan 2026 04:57:20 GMT', 'content-type': 'application/json', 'content-length': '307', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '1', 'x-pinecone-request-latency-ms': '34', 'x-pinecone-request-id': '247806141954132923', 'x-envoy-upstream-service-time': '35', 'x-pinecone-response-duration-ms': '36', 'grpc-status': '0', 'server': 'envoy'}})


In [ ]:
import time

# ---------------------------------------------------------
# Queries to compare (Before vs After)
# ---------------------------------------------------------
queries = {
    "Sentence 1": "data engineering resume, azure data factory, azure databricks",
    "Sentence 2": "data science machine learning langchain gen ai agentic ai"
}

results_table = []

# ---------------------------------------------------------
# Run Performance Benchmark
# ---------------------------------------------------------
for label, query_text in queries.items():
    print(f"\nRunning {label} query...")

    # Embedding timing
    embed_start = time.time()
    query_embedding = model.encode(query_text).tolist()
    embed_end = time.time()

    # Pinecone query timing
    search_start = time.time()
    results = index.query(
        vector=query_embedding,
        top_k=5,
        include_values=False
    )
    search_end = time.time()

    embedding_time = embed_end - embed_start
    pinecone_time = search_end - search_start
    total_time = search_end - embed_start

    results_table.append({
        "Query Type": label,
        "Embedding Time (s)": round(embedding_time, 4),
        "Pinecone Query Time (s)": round(pinecone_time, 4),
        "Total Time (s)": round(total_time, 4)
    })

# ---------------------------------------------------------
# Print Comparison Table
# ---------------------------------------------------------
print("\n================ PERFORMANCE COMPARISON ================\n")
print(f"{'Query':<10} | {'Embedding (s)':<14} | {'Pinecone (s)':<14} | {'Total (s)':<10}")
print("-" * 60)

for row in results_table:
    print(
        f"{row['Query Type']:<10} | "
        f"{row['Embedding Time (s)']:<14} | "
        f"{row['Pinecone Query Time (s)']:<14} | "
        f"{row['Total Time (s)']:<10}"
    )

print("\n========================================================\n")


Running Sentence 1 query...

Running Sentence 2 query...

================ PERFORMANCE COMPARISON ================

Query      | Embedding (s)  | Pinecone (s)   | Total (s) 
------------------------------------------------------------
Sentence 1 | 0.12           | 0.0709         | 0.1909    
Sentence 2 | 0.0272         | 0.028          | 0.0552    




In [ ]:
import time

QUERY_TEXT = "azure data engineer with databricks and adf"
RESULTS = []

# ---------------------------------------------------------
# Utility function to measure time
# ---------------------------------------------------------
def benchmark(method_name, embed_fn, query_fn):
    embed_start = time.time()
    vector = embed_fn()
    embed_end = time.time()

    search_start = time.time()
    query_fn(vector)
    search_end = time.time()

    RESULTS.append({
        "Method": method_name,
        "Embedding Time (s)": round(embed_end - embed_start, 4),
        "Pinecone Time (s)": round(search_end - search_start, 4),
        "Total Time (s)": round(search_end - embed_start, 4)
    })


# ---------------------------------------------------------
# METHOD 1: BASELINE (No optimization)
# ---------------------------------------------------------
benchmark(
    "Baseline",
    lambda: model.encode(QUERY_TEXT).tolist(),
    lambda v: index.query(vector=v, top_k=5, include_values=False)
)


# ---------------------------------------------------------
# METHOD 2: WARM-UP QUERY (Index & model warm)
# ---------------------------------------------------------
_ = model.encode(QUERY_TEXT)
_ = index.query(vector=model.encode(QUERY_TEXT).tolist(), top_k=5)

benchmark(
    "Warm-up",
    lambda: model.encode(QUERY_TEXT).tolist(),
    lambda v: index.query(vector=v, top_k=5, include_values=False)
)


# ---------------------------------------------------------
# METHOD 3: EMBEDDING REUSE (Cached embedding)
# ---------------------------------------------------------
cached_embedding = model.encode(QUERY_TEXT).tolist()

benchmark(
    "Embedding Reuse",
    lambda: cached_embedding,
    lambda v: index.query(vector=v, top_k=5, include_values=False)
)


# ---------------------------------------------------------
# METHOD 4: REDUCED TOP_K
# ---------------------------------------------------------
benchmark(
    "Reduced top_k",
    lambda: model.encode(QUERY_TEXT).tolist(),
    lambda v: index.query(vector=v, top_k=3, include_values=False)
)


# ---------------------------------------------------------
# METHOD 5: VECTOR REUSE + REDUCED TOP_K (FASTEST)
# ---------------------------------------------------------
benchmark(
    "Vector + top_k Reuse",
    lambda: cached_embedding,
    lambda v: index.query(vector=v, top_k=3, include_values=False)
)


# ---------------------------------------------------------
# PRINT PERFORMANCE COMPARISON TABLE
# ---------------------------------------------------------
print("\n================ PERFORMANCE BENCHMARK =================\n")
print(f"{'Method':<22} | {'Embedding (s)':<14} | {'Pinecone (s)':<14} | {'Total (s)':<10}")
print("-" * 70)

for r in RESULTS:
    print(
        f"{r['Method']:<22} | "
        f"{r['Embedding Time (s)']:<14} | "
        f"{r['Pinecone Time (s)']:<14} | "
        f"{r['Total Time (s)']:<10}"
    )

print("\n========================================================\n")


================ PERFORMANCE BENCHMARK =================

Method                 | Embedding (s)  | Pinecone (s)   | Total (s) 
----------------------------------------------------------------------
Baseline               | 0.172          | 0.4456         | 0.6177    
Warm-up                | 0.0301         | 0.0301         | 0.0602    
Embedding Reuse        | 0.0            | 0.0315         | 0.0315    
Reduced top_k          | 0.0257         | 0.0285         | 0.0542    
Vector + top_k Reuse   | 0.0            | 0.0296         | 0.0296    


